## Setup

In [ ]:
!pip uninstall transformers peft accelerate trl bitsandbytes datasets huggingface-hub safetensors pandas matplotlib numpy torch

Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.46.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.46.2
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/peft-0.13.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/peft/*
Proceed (Y/n)? Y
  Successfully uninstalled peft-0.13.2
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/bin/accelerate-merge-weights
    /usr/local/lib/python3.11/dist-packages/accelerate-1.1.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/acce

In [1]:
!pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
import torch

In [ ]:
from huggingface_hub import login

login()

## Loading the model

In [3]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(
    repo_id, device_map="cuda:0", quantization_config=bnb_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072

In [ ]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,                   # the rank of the adapter, the lower the fewer parameters you'll need to train
    lora_alpha=16,         # multiplier, usually 2*r
    bias="none",           # BEWARE: training biases *modifies* base model's behavior
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # Newer models, such as Phi-3 at time of writing, may require
    # manually setting target modules
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)

model = get_peft_model(model, config)

## Preparing the data

In [ ]:
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [ ]:
def prep_dataset(items):
  converted_sample = [
      {"role": "user", "content": items["sql_prompt"]},
      {"role": "assistant", "content": items["sql"]},
      ]

  return {'messages': converted_sample}

In [ ]:
prepped_dataset = dataset.map(prep_dataset).remove_columns([
    'id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'])
prepped_dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 100000
})

In [12]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

## Fine-tuning the model

In [ ]:
subdataset = prepped_dataset.select(range(5000))

In [ ]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True,
    # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    gradient_accumulation_steps=1,
    per_device_train_batch_size=64,
    # If batch size would cause OOM, halves its size until it works
    auto_find_batch_size=True,

    ## GROUP 2: Dataset-related
    max_seq_length=64,
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,

    num_train_epochs=2,
    learning_rate=3e-4,
    # Optimizer
    # 8-bit Adam optimizer
    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=5,
    logging_dir='./logs',
    output_dir='./phi3-mini-texttosql-adapter-no_instrction',
    report_to='none'
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=subdataset,
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [ ]:
trainer.train()

Step,Training Loss
5,1.137800
10,1.115800
15,1.090600
20,1.118000
25,1.089900
30,1.070300
35,1.085300
40,1.071800
45,1.071700
50,1.055500


TrainOutput(global_step=160, training_loss=1.00310360789299, metrics={'train_runtime': 2061.213, 'train_samples_per_second': 4.954, 'train_steps_per_second': 0.078, 'total_flos': 1.4647093583413248e+16, 'train_loss': 1.00310360789299, 'epoch': 2.0})

## Registry

### Uploading the adapter

In [ ]:
trainer.save_model('local-phi3-mini-texttosql-adapter-no_instrction')

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/szaboildi/phi3-mini-texttosql-adapter-no_instrction/commit/23005ae24d7502e625fd2812fc4ff731fc33148a', commit_message='End of training', commit_description='', oid='23005ae24d7502e625fd2812fc4ff731fc33148a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/szaboildi/phi3-mini-texttosql-adapter-no_instrction', endpoint='https://huggingface.co', repo_type='model', repo_id='szaboildi/phi3-mini-texttosql-adapter-no_instrction'), pr_revision=None, pr_num=None)

### Merging the adapter and the model

In [ ]:
base_model_name_or_path = "microsoft/Phi-3-mini-4k-instruct"
peft_model_path = "szaboildi/phi3-mini-texttosql-adapter-no_instrction"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    return_dict=True,
    torch_dtype=torch.float32,
    device_map="cuda:0"
)

model = PeftModel.from_pretrained(base_model, peft_model_path)
model_merged = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
merged_model_name = "phi3-mini-4k-instruct-texttosql-merged-noinstruction"
model_merged.save_pretrained(merged_model_name)
tokenizer.save_pretrained(merged_model_name)

('phi3-mini-4k-instruct-texttosql-merged-noinstruction/tokenizer_config.json',
 'phi3-mini-4k-instruct-texttosql-merged-noinstruction/special_tokens_map.json',
 'phi3-mini-4k-instruct-texttosql-merged-noinstruction/tokenizer.model',
 'phi3-mini-4k-instruct-texttosql-merged-noinstruction/added_tokens.json',
 'phi3-mini-4k-instruct-texttosql-merged-noinstruction/tokenizer.json')

In [ ]:
print(f"Saving to hub ...")
model_merged.push_to_hub(merged_model_name, use_temp_dir=False, private=False)
tokenizer.push_to_hub(merged_model_name, use_temp_dir=False, private=False)

Saving to hub ...


Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction/commit/5813b2589ca69070f233b618aa531951f6e6dea3', commit_message='Upload tokenizer', commit_description='', oid='5813b2589ca69070f233b618aa531951f6e6dea3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction', endpoint='https://huggingface.co', repo_type='model', repo_id='szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction'), pr_revision=None, pr_num=None)

### GGUF

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
# model_dl = AutoModelForCausalLM.from_pretrained(
#     "szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction",
#     device_map="cuda:0")
# tokenizer_dl = AutoTokenizer.from_pretrained("szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction")

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 51212, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 51212 (delta 91), reused 43 (delta 43), pack-reused 51066 (from 4)
Receiving objects: 100% (51212/51212), 117.03 MiB | 36.06 MiB/s, done.
Resolving deltas: 100% (37006/37006), done.


In [ ]:
!pip install -r llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py phi3-mini-4k-instruct-texttosql-merged-noinstruction --outfile phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_q8.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: phi3-mini-4k-instruct-texttosql-merged-noinstruction
INFO:hf-to-gguf:Model architecture: Phi3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.float32 --> Q8_0, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float32 --> Q8_0, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float32 --> Q8_0, shape = {3072, 16384}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.float32 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_qkv.weight,     torch.float32 --> Q8_0, shape = {3072, 921

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py phi3-mini-4k-instruct-texttosql-merged-noinstruction --outfile phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_f16.gguf --outtype f16

INFO:hf-to-gguf:Loading model: phi3-mini-4k-instruct-texttosql-merged-noinstruction
INFO:hf-to-gguf:Model architecture: Phi3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.float32 --> F16, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float32 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float32 --> F16, shape = {3072, 16384}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.float32 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_qkv.weight,     torch.float32 --> F16, shape = {3072, 9216}
IN

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_q8.gguf",
    path_in_repo="phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_q8.gguf",
    repo_id=model_id,
)

api.upload_file(
    path_or_fileobj="phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_f16.gguf",
    path_in_repo="phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_f16.gguf",
    repo_id=model_id,
)

phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_q8.gguf:   0%|          | 0.00/4.06G [00:00<?, ?B/s]

phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_f16.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s…

CommitInfo(commit_url='https://huggingface.co/szaboildi/phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf/commit/614572bf1618a2ee4276acbb0bf5c618148c2331', commit_message='Upload phi3-mini-4k-instruct-texttosql-merged-noinstruction_gguf_f16.gguf with huggingface_hub', commit_description='', oid='614572bf1618a2ee4276acbb0bf5c618148c2331', pr_url=None, pr_revision=None, pr_num=None)

## Testing

In [7]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [8]:
sentence = "How many needles are in the three biggest haystacks?"
sentence2 = "What colors were apples from the last five months?"

In [9]:
import re

def format_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "user", "content": sentence},
    ]
    prompt = tokenizer.apply_chat_template(converted_sample,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt

In [18]:
def generate(model, tokenizer, query, max_new_tokens=128, skip_special_tokens=False):
    prompt = format_prompt(tokenizer, query)

    tokenized_input = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

    model.eval()
    generation_output = model.generate(**tokenized_input,
                                       eos_token_id=tokenizer.eos_token_id,
                                       max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(generation_output,
                                    skip_special_tokens=skip_special_tokens)
    reply = re.search('<\|assistant\|>(.*?)<\|end\|><\|endoftext\|>', output[0])

    if reply is None:
        return output[0]
    return reply.group(1)

In [ ]:
print(generate(model, tokenizer, sentence))

SELECT SUM(needles) FROM haystacks WHERE id IN (SELECT id FROM haystacks ORDER BY size DESC LIMIT 3);


In [ ]:
print(generate(model_merged, tokenizer, sentence))

SELECT SUM(needles) FROM haystacks WHERE id IN (SELECT id FROM haystacks ORDER BY size DESC LIMIT 3);


In [ ]:
print(generate(model, tokenizer, sentence2))

SELECT DISTINCT color FROM fruits WHERE name = 'apple';


In [4]:
model_og = AutoModelForCausalLM.from_pretrained(
    repo_id, device_map="cuda:0", quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
print(generate(model_og, tokenizer, sentence))

 I'm sorry, but I cannot provide a specific answer to the number of needles in the three biggest haystacks without additional context or information. The number of needles would depend on the size of the haystacks and the density of the needles within them. If you can provide more details or clarify the context of this question, I might be able to assist you better.


In [20]:
print(generate(model_og, tokenizer, sentence2))

 I'm sorry, but I don't have the capability to access real-time or historical data on the color of apples from the last five months. Apple colors can vary widely depending on the variety and ripeness. For instance, some apples may be green when unripe and turn red, yellow, or even pink as they ripen. To find out the colors of apples from the last five months, you would need to consult a local farmer, a grocery store, or an agricultural database that tracks produce.
